# Velocity Data Download Script for Validation
#### Working Script to pull large velocity datasets and later compare to ICESAT2-derived velocities

ICESat-2 hackweek  
June 15, 2020  
Lynn Kaluzienski

# Import necessary modules

In [1]:
import os,re,h5py
import requests
import zipfile

In [2]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
import scipy, sys, os, pyproj, glob
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import pointCollection as pc

# Import some of the scripts that we have written
import sys
sys.path.append("/home/jovyan/surface_velocity/scripts")
from loading_scripts import atl06_to_dict

# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [3]:
#From Ben Smith's code loading in .tif file, running into issues likely with directories
data_root='/srv/shared/surface_velocity/FIS_Velocity/'
#spatial_extent = np.array([-102, -76, -98, -74.5])
spatial_extent = np.array([-65, -86, -55, -81])
lat=spatial_extent[[1, 3, 3, 1, 1]]
lon=spatial_extent[[2, 2, 0, 0, 2]]
print(lat)
print(lon)
# project the coordinates to Antarctic polar stereographic
xy=np.array(pyproj.Proj(3031)(lon, lat))
# get the bounds of the projected coordinates 
XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
#Originally tried to load data from a local directory, should change to shared directory
Measures_vx=pc.grid.data().from_geotif(os.path.join(data_root,'Measures2_FIS_Vx.tif'), bounds=[XR, YR])
Measures_vy=pc.grid.data().from_geotif(os.path.join(data_root,'Measures2_FIS_Vy.tif'), bounds=[XR, YR])

[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]


In [4]:
# Load a line and plot

data_root='/srv/shared/surface_velocity/'
field_dict={None:['delta_time','latitude','longitude','h_li', 'atl06_quality_summary'],\
                    'ground_track':['x_atc','y_atc'],\
                    'fit_statistics':['dh_fit_dx', 'dh_fit_dy']}

rgt = "0848"
cycle="03"
filename = glob.glob(os.path.join(data_root, 'FIS_ATL06', f'*ATL06_*_{rgt}{cycle}*_003*.h5'))[0]

D=atl06_to_dict(filename,'/gt2l', field_dict=field_dict, index=None, epsg=3031)

# show the velocity map:
plt.figure(figsize=(8,4))
plt.subplot(121)
Measures_vx.show(cmap='viridis', clim=[-100,100])
plt.plot(xy[0,:], xy[1,:],'k')
plt.title('Measures X-Velocity')
plt.plot(D['x'],D['y'],'r')
plt.gca().set_aspect('equal')

plt.subplot(122)
Measures_vy.show(cmap='viridis', clim=[-100,100])
plt.plot(xy[0,:], xy[1,:],'k')
plt.title('Measures Y-Velocity')
plt.plot(D['x'],D['y'],'r')
plt.gca().set_aspect('equal')

#plt.tight_layout()

# Interpolate the Measures velocities along the line and plot (note that these are speeds for now)

vx = Measures_vx.interp(D['x'],D['y'])
vy = Measures_vy.interp(D['x'],D['y'])

plt.figure(figsize=(8,4))
plt.subplot(121)
plt.plot(D['x_atc'],vx)
plt.title('X-Velocity')
plt.subplot(122)
plt.plot(D['x_atc'],vy)
plt.title('Y-Velocity')

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

{'cmap': 'viridis', 'clim': [-100, 100], 'extent': array([-887878.65809562, -400567.81840096,  200333.90920048,
        561654.87344315]), 'origin': 'lower'}
{'cmap': 'viridis', 'clim': [-100, 100], 'extent': array([-887878.65809562, -400567.81840096,  200333.90920048,
        561654.87344315]), 'origin': 'lower'}


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# add measures along track to dictionary


In [40]:

# should be close to 20
hypoth = np.sqrt((D['x'][0]-D['x'][1])**2+(D['y'][0]-D['y'][1])**2)
print("Lenght of the first segment is ", hypoth , " this is close to the teoretical elngth of 20 m")

adjacent = abs(D['x'][0]-D['x'][1])

angle = np.arccos(adjacent/hypoth)

print("The angle of the track for vel transfromation is: " ,angle, " radians")

Lenght of the first segment is  19.47695252813686  this is close to the teoretical elngth of 20 m
The angle of the track for vel transfromation is:  0.8401541611474018  radians


In [ ]:

vx = Measures_vx.interp(D['x'],D['y'])
vy = Measures_vy.interp(D['x'],D['y'])
